In [7]:
from ultralytics import YOLO
import cv2
import numpy as np

def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        fruit_roi = img[y1:y2, x1:x2]
        color = detect_color(fruit_roi)  # Modified color detection
        
        fruits.append({
            "view": view,
            "bbox": (x1, y1, x2, y2),
            "center": center,
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find dominant color using HSV masking."""
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    height, width = hsv.shape[:2]
    
    # Define color ranges (H_min, S_min, V_min), (H_max, S_max, V_max)
    color_masks = {
        "Red": [
            ((0, 100, 100), (10, 255, 255)),
            ((160, 100, 100), (180, 255, 255))
        ],
        "Orange": [((11, 150, 100), (25, 255, 255))],
        "Yellow": [((26, 150, 100), (35, 255, 255))],
        "Green": [((36, 100, 100), (85, 255, 255))],
        "Blue": [((86, 100, 100), (124, 255, 255))],
        "Purple": [((125, 50, 50), (159, 255, 255))]  # Wider purple range
    }

    dominant_color = "Unknown"
    max_pixels = 0
    
    for color_name, ranges in color_masks.items():
        combined_mask = np.zeros((height, width), dtype=np.uint8)
        
        # Combine all ranges for the color
        for (lower, upper) in ranges:
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(hsv, lower, upper)
            combined_mask = cv2.bitwise_or(combined_mask, mask)
        
        # Clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Count valid pixels
        pixel_count = cv2.countNonZero(cleaned_mask)
        if pixel_count > max_pixels and pixel_count > 0.1 * (height * width):
            max_pixels = pixel_count
            dominant_color = color_name
            
    return dominant_color

def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]  # Take best matches
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([[f["center"]] for f in back_fruits])  # Use center instead of bbox
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_center = np.array(front["center"])
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                dist = np.linalg.norm(front_center - trans_center)
                if dist < 30:  # Keep reasonable distance threshold
                    matched_back.add(i)
                    matched = True
                    break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Front3.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Back3.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")



image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Front3.jpg: 480x640 4 1s, 1 3, 258.9ms
Speed: 2.2ms preprocess, 258.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\3\Back3.jpg: 480x640 4 1s, 1 3, 267.9ms
Speed: 2.0ms preprocess, 267.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Front, Color: Yellow
View: Back, Color: Yellow

Total unique fruits detected: 2


In [3]:
import shutil

# Zip the entire runs directory
shutil.make_archive('runs', 'zip', 'runs')
print("Runs directory zipped as runs.zip")
from IPython.display import FileLink

# Create a download link for the zipped file
FileLink('runs.zip')


Runs directory zipped as runs.zip


C:\Users\saart\runs.zip

In [53]:
def detect_fruits(img_path, model, view):
    """Detect fruits in an image, find their colors, and keep track of which view they belong to."""
    results = model.predict(img_path)[0]
    fruits = []
    img = cv2.imread(img_path)
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        fruit_roi = img[y1:y2, x1:x2]
        color = detect_color(fruit_roi)  # Modified color detection
        
        fruits.append({
            "view": view,
            "bbox": (x1, y1, x2, y2),
            "center": center,
            "color": color
        })
    return fruits

def detect_color(roi):
    """Find dominant color using HSV masking."""
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    height, width = hsv.shape[:2]
    
    # Define color ranges (H_min, S_min, V_min), (H_max, S_max, V_max)
    color_masks = {
        "Red": [
            ((0, 100, 100), (10, 255, 255)),
            ((160, 100, 100), (180, 255, 255))
        ],
        "Orange": [((11, 150, 100), (25, 255, 255))],
        "Yellow": [((26, 150, 100), (35, 255, 255))],
        "Green": [((36, 100, 100), (85, 255, 255))],
        "Blue": [((86, 100, 100), (124, 255, 255))],
        "Purple": [((125, 50, 50), (159, 255, 255))]  # Wider purple range
    }

    dominant_color = "Unknown"
    max_pixels = 0
    
    for color_name, ranges in color_masks.items():
        combined_mask = np.zeros((height, width), dtype=np.uint8)
        
        # Combine all ranges for the color
        for (lower, upper) in ranges:
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(hsv, lower, upper)
            combined_mask = cv2.bitwise_or(combined_mask, mask)
        
        # Clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Count valid pixels
        pixel_count = cv2.countNonZero(cleaned_mask)
        if pixel_count > max_pixels and pixel_count > 0.1 * (height * width):
            max_pixels = pixel_count
            dominant_color = color_name
            
    return dominant_color

def align_views(front_img, back_img):
    """Aligns the front and back views using feature matching."""
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(front_img, None)
    kp2, des2 = orb.detectAndCompute(back_img, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:50]  # Take best matches
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    return H

def count_unique_fruits(front_path, back_path, model_path):
    """Count the number of unique fruits seen from both front and back views."""
    model = YOLO(model_path)
    front_img = cv2.imread(front_path)
    back_img = cv2.imread(back_path)
    
    front_fruits = detect_fruits(front_path, model, "Front")
    back_fruits = detect_fruits(back_path, model, "Back")
    
    H = align_views(front_img, back_img)
    
    back_centers = np.float32([[f["center"]] for f in back_fruits])  # Use center instead of bbox
    transformed_centers = cv2.perspectiveTransform(back_centers, H).reshape(-1, 2) if len(back_centers) > 0 else []
    
    matched_back = set()
    unique_fruits = []
    
    for front in front_fruits:
        front_center = np.array(front["center"])
        front_color = front["color"]
        matched = False
        
        for i, trans_center in enumerate(transformed_centers):
            if i not in matched_back and front_color == back_fruits[i]["color"]:
                dist = np.linalg.norm(front_center - trans_center)
                if dist < 30:  # Keep reasonable distance threshold
                    matched_back.add(i)
                    matched = True
                    unique_fruits.append({"view": "Both", "color": front_color})
                    break
        
        if not matched:
            unique_fruits.append(front)
    
    for i in range(len(back_fruits)):
        if i not in matched_back:
            unique_fruits.append(back_fruits[i])
    
    print("\nUnique Fruits and Their Colors:")
    for fruit in unique_fruits:
        print(f"View: {fruit['view']}, Color: {fruit['color']}")
    
    return len(unique_fruits)

if __name__ == "__main__":
    front_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg'
    back_path = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg'
    model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
    
    total_fruits = count_unique_fruits(front_path, back_path, model_path)
    print(f"\nTotal unique fruits detected: {total_fruits}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg: 480x640 5 2s, 3 3s, 275.8ms
Speed: 4.0ms preprocess, 275.8ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg: 480x640 8 2s, 3 3s, 188.0ms
Speed: 6.5ms preprocess, 188.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

Unique Fruits and Their Colors:
View: Both, Color: Green
View: Both, Color: Green
View: Both, Color: Green
View: Front, Color: Purple
View: Front, Color: Purple
View: Both, Color: Purple
View: Both, Color: Purple
View: Both, Color: Purple
View: Back, Color: Purple
View: Back, Color: Purple
View: Back, Color: Purple
View: Back, Color: Purple
View: Back, Color: Purple

Total unique fruits detected: 13
